In [54]:
import threading
import mysql.connector
from mysql.connector import Error, errorcode
import time # это для таймера
from datetime import datetime


class Bd_rabota():
  def __init__(self, reg):
    print(f"{datetime.now()}| cls_bd.INIT(idO={reg}): инициализация объекта БД_работа.")
    self.__host='localhost'         # например, 'localhost' или IP адрес сервера
    self.__database='diser' # имя вашей базы данных
    self.__user='forpiton'     # имя пользователя
    self.__password='passwordpiton'   # пароль
    self.__connection = False #само соединение с СУБД. предполагается держать открытым
    self.__oregno = reg  #идентификатор для многопоточной работы
    #print(errorcode.ER_DUP_ENTRY, 'ddddddd')

  def conn(self): #коннект воркера к бд
    print(f"{datetime.now()}| cls_bd.CONNECT(idO={self.__oregno}): метод коннекта к бд начало.")
    if self.__connection:
      kod_log = self.__subd.add_log(self.__class__.__name__,'conn',200,'пытались выполнить метод connect, но коннект то уже есть. выходим')
      return 200
      
    try:
            self.__connection = mysql.connector.connect(host=self.__host,
                database=self.__database,
                user=self.__user,
                password=self.__password)
            if self.__connection.is_connected():
                print(f"  {datetime.now()}| cls_bd.CONNECT(idO={self.__oregno}): Успешно подключено к базе данных")
    except Error as e:
            print(f"  {datetime.now()}| cls_bd.CONNECT(idO={self.__oregno}): Ошибка в методе CONN при подключении к MySQL: {e}")

  def disc(self): #дисконнект воркера к бд
    print(f"{datetime.now()}| cls_bd.DISCONNECT(idO={self.__oregno}): метод дисконнекта от бд начало, conn= {self.__connection}")
    try:
#      if self.__connection.is_connected():
      if self.__connection:
          self.__connection.close()
          print(f"  {datetime.now()}| cls_bd.DISCONNECT(idO={self.__oregno}): Соединение с MySQL было закрыто")
      else:
        print(f'  {datetime.now()}| cls_bd.DISCONNECT(idO={self.__oregno}):в методе дисконнект на найдено открытого подключения к субд')
    except Error as e:
      print(f"{datetime.now()}| cls_bd.DISCONNECT(idO={self.__oregno}) Ошибка в методе DISCONNECT при отключении от MySQL: {e}")

  def day_rows(self, select_txt):
    print(f'cls_bd.DAY_ROWS(idO={self.__oregno}): зашел в процедуру day_cursor')
    try:
      if self.__connection.is_connected():
        print(f'  cls_bd.DAY_ROWS(idO={self.__oregno}): живой коннект обнаружен')
        curs = self.__connection.cursor()
        # Выполнение запроса SELECT
        curs.execute(select_txt)
        print(f'  cls_bd.DAY_ROWS(idO={self.__oregno}): select выполнил, возвращаю LIST ')
        recs = curs.fetchall()  # Получение всех результатов запроса. обязательно выполнять, иначе is_connected() будет возвращать FALSE
        curs.close() # закрыть сразу
        return recs
      else:
        print(f'  cls_bd.DAY_ROWS(idO={self.__oregno}): живого коннекта не обнаружено, возвращать нечего')
    except Error as e:
        print(f"cls_bd.DAY_ROWS(idO={self.__oregno}): Ошибка в методе DAY_ROWS(): {e}")
  
  def ins_url(self, uurl, uurlred, iskl, kod, status,lim_rcs, lim_hrf, lim_sz, sz, errortext,recurs_lvl):
    print(f'{datetime.now()}| cls_bd.INSERT(idO={self.__oregno}): метод INSERT записи в таблицу. урл={uurl}, status={status}')
    error_kod = 0
    try:
      if self.__connection.is_connected():
        print(f'  {datetime.now()}| cls_bd.INSERT(idO={self.__oregno}): живой коннект обнаружен')
        cursor_insert = self.__connection.cursor()
        #insert_txt = f'insert into t_url (url, url_red, url_id_iskl, kod_otveta, busy, status, limit_recurs, limit_href, limit_size, url_size, err_txt) values ("{uurl}","{uurlred}",{iskl},{kod},{self.__oregno}, {status}, {lim_rcs}, {lim_hrf}, {lim_sz}, {sz},"{errortext}")'
        insert_txt = f'''
        insert into t_url set url="{uurl}", url_red="{uurlred}", url_id_iskl={iskl}, kod_otveta={kod}, busy={self.__oregno}, status={status}, 
        limit_recurs={lim_rcs}, limit_href={lim_hrf}, limit_size={lim_sz}, url_size={sz}, err_txt="{errortext}", kolwo_recurs={recurs_lvl}'''

        print(f'  {datetime.now()}| cls_bd.INSERT(idO={self.__oregno}): после сборки insert={insert_txt}')
        try:
          cursor_insert.execute(insert_txt)
          self.__connection.commit()
        except Error as e:
          print(f"    {datetime.now()}| cls_bd.INSERT(idO={self.__oregno}): Ошибка в методе INS_URL(). insert сломался: {e}, код ошибки: {e.errno}")  
          error_kod = e.errno
      else:
        print(f'  {datetime.now()}| cls_bd.INSERT(idO={self.__oregno}): живого коннекта не обнаружено, на всякий случай остановлюсь')  
        return 'stop'
        
    except Error as e:
        print(f"  {datetime.now()}| cls_bd.INSERT(idO={self.__oregno}): Ошибка в методе INS_URL(): {e}, код ошибки: {e.errno}")
        error_kod = e.errno
    finally:
      if cursor_insert:
          cursor_insert.close()
          print(f"  {datetime.now()}| cls_bd.INSERT(idO={self.__oregno}): Mетод INS_URL(), успешно закрыт курсор")
      else:
          print(f"  {datetime.now()}| cls_bd.INSERT(idO={self.__oregno}): Ошибка в методе INS_URL()? при закрытии курсора")
      print( f'{datetime.now()}| сейчас буду выходить из ins_url c кодом выхода {error_kod}')    
      return error_kod

  def add_log(self, klass, metod, level, txt): #запись в лог-таблицу
    error_kod = 0
    insert_txt =''
    if not self.__connection:
      error_kod = 20010
      raise Log20010(f'коннект к БД не найден в классе {self.__class__.__name__}, метод add_log', error_kod)
    try:
      curs_add_log = self.__connection.cursor()
      insert_txt = f'insert into t_log set class="{klass}", metod="{metod}", level={level}, worker_nom={self.__oregno}, txt="{txt}"'
      try:
        curs_add_log.execute(insert_txt)
        self.__connection.commit()
      except Exception as e:
        error_kod = e #не смог добавить запись в лог или сделать коммит
    except Exception as e:
      error_kod = e #не смог создать курсор
    finally:
      if curs_add_log:
        curs_add_log.close()
      else:
        print('чето непонятное. попытался закрыть курсор curs_add_log, но он не найден')
    return error_kod
    
  def test_conn(self):
    print(f'cls_bd.TEST_CONN(idO={self.__oregno}): зашел в процедуру test_conn')
    try:
      if self.__connection.is_connected():
        print(f'  cls_bd.TEST_CONN(idO={self.__oregno}): живой коннект обнаружен')
      else:
        print(f'  cls_bd.TEST_CONN(idO={self.__oregno}): живого коннекта не обнаружено, возвращать нечего')
    except Error as e:
        print(f"cls_bd.TEST_CONN(idO={self.__oregno}): Ошибка в методе TEST_CONN: {e}")    
      
      
  def inf(self): # инфо
    print(f'cls_bd.INF(idO={self.__oregno}): информация о переменных. Хост=', self.__host,  ' дб=',self.__database, ' юзер=',self.__user, ' пароль=',self.__password, ' regno=',self.__oregno, ' connection=',self.__connection)
    if self.__connection.is_connected():
      print(f'  cls_bd.INF(idO={self.__oregno}): connect обнаружен')
    else:
      print(f'  cls_bd.INF(idO={self.__oregno}): connect НЕ обнаружен')

  def day_svoy_nomer(self):
    return self.__oregno
# ^^^^^^^^^^^^^^^^^^^^^^^ конец класса Bd_rabota ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [74]:
import random
def worker():
  #sl = 60
  
  #print("WORKER(): зашли в воркер")
  kod_vyhoda = 'Ok'
  regno = random.randrange(0, 9999)
  #print(f'WORKER(): присвоили ID={regno}')
  try:
    wsubd = Bd_rabota(regno)
    wsubd.conn()
    wkod_log = wsubd.add_log('Worker','begin',1,f'выполнили CONNECT')
    if wkod_log != 0:
      return f'попытка записи в лог вернула ошибку {wkod_log}'
  except Exception as e:
    print('не смог победить бд, текст ошибки:',e)
    return 1
 
  kr = Krauler( 'https://bamboo.phpbb.com/allPlans.action', wsubd)
  #kr = Krauler(regno, 'http://www.facebook.com/phpBB')
  #kr = Krauler(regno, 'https://biblio1.ru/test_diser1.html')
  err_kod=kr.start1()
  kod_log = wsubd.add_log('Worker','вышел из start1',100,f'вышел с кодом:{err_kod}')
  if kod_log !=0:
    print('ошибка при записи в лог из Worker в конце. ошибка-',kod_log)
    return 20000 #работать не можем дальше, останавливаемся, нужно разбираться

  try:
    wkod_log = wsubd.add_log('Worker','end',1000,f'еще буду пытаться сделать disconnect')
    wsubd.disc()
  except Exception as e:
    print(f'Останов! не смог отключиться от БД, {e}')
    return 3
  return kod_vyhoda

worker()

2025-03-22 20:48:41.188550| cls_bd.INIT(idO=8022): инициализация объекта БД_работа
2025-03-22 20:48:41.188550| cls_bd.CONNECT(idO=8022): метод коннекта к бд начало.
  2025-03-22 20:48:41.308755| cls_bd.CONNECT(idO=8022): Успешно подключено к базе данных
2025-03-22 20:48:42.165795| cls_bd.INSERT(idO=8022): метод INSERT записи в таблицу. урл=https://bamboo.phpbb.com/allPlans.action, status=1
  2025-03-22 20:48:42.165795| cls_bd.INSERT(idO=8022): живой коннект обнаружен
  2025-03-22 20:48:42.166796| cls_bd.INSERT(idO=8022): после сборки insert=
        insert into t_url set url="https://bamboo.phpbb.com/allPlans.action", url_red="", url_id_iskl=0, kod_otveta=200, busy=8022, status=1, 
        limit_recurs=10, limit_href=40, limit_size=1048576, url_size=56408, err_txt="", kolwo_recurs=0
  2025-03-22 20:48:42.170793| cls_bd.INSERT(idO=8022): Mетод INS_URL(), успешно закрыт курсор
2025-03-22 20:48:42.170793| сейчас буду выходить из ins_url c кодом выхода 0
2025-03-22 20:48:42.199209| cls_kr.

'Ok'

In [72]:
import requests
from requests.exceptions import HTTPError
import random
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlsplit, urlunsplit

class Log20010(Exception): #нужно будет просто выйти
  def __init__(self,mes,kod):
    self.myerrno = kod
  pass

class NNN(Exception):
  pass
  #def log_and_exit()

class Krauler(): 
  def __init__(self, start_url, psubd):
    self.__subd = psubd #один объект Bd_rabota на один объект краулера. типа это пользователь. не нужно беспокоиться о конкурентном доступе при многопоточной работе краулеров. СУБД сама разрулит одновременный доступ от разных краулеров. пул коннектов решил не делать,т.к. поток всегда чтото пишет.вроде логично держать один коннект на один поток/воркер
    self.__krregno = self.__subd.day_svoy_nomer()
    kod_log = self.__subd.add_log(self.__class__.__name__,'init',1,'инициализация объекта Krauler. третья строка кода1111111111111')
    if kod_log !=0:
      print('ошибка при записи в лог из конструктора краулера. ошибка-',kod_log)
      return 20000

    
#    try:
#      self.__subd.add_log(self.__class__.__name__,'start1',1,f'Начало метода START1 в Krauler. стартую с урла:{self.__start_url}')
#    except Log_20000:
#      return 20000
    
    self.__start_url = start_url  # откуда начинаем
    self.__max_count = 15   # максимальное количество скачиваний урлов. сквозная относительно вложенности и количества урлов на странице
    self.__now_count = 0   # текущее количество скачиваний урлов. сквозная относительно вложенности и количества урлов на странице
    #self.__visited = set()          # посещенные страницы
    self.__setup_lim_recurs = 10 #глубина вложенности. на сколько в рекурсию можем провалиться одним воркером
    self.__now_recurs = 0 #глубина вложенности. на текущий момент
    self.__setup_lim_href = 40 #максимальное количество ссылок на странице. сверх этого не обрабатываем
    self.__setup_lim_size = 1048576 #максимальный размер ответа в байтах, который обрабатываем
    self.__urldubley = 0 #здесь можно считать дубли, которые будут пытаться записываться в базу. для дальнейшего анализа
    
  def start1(self):
    kod_log = self.__subd.add_log(self.__class__.__name__,'start1',1,f'Начало метода START1 в Krauler. стартую с урла:{self.__start_url}')
    if kod_log !=0:
      print('ошибка при записи в лог из Кrauler.start1 ошибка-',kod_log)
      return 20000 #работать не можем дальше, останавливаемся, нужно разбираться

    kod_dl  = self.__download_link(self.__start_url, 1)
    kod_log = self.__subd.add_log(self.__class__.__name__,'start1',1,f'Закончил метод START1 в Krauler с кодом {kod_dl}, дублей всего было {self.__urldubley}')
    if kod_log !=0:
      print('ошибка при записи в лог из краулера. START1 lvl=1. ошибка-',kod_log)
      return 20000 #работать не можем дальше, останавливаемся, нужно разбираться
    return kod_dl

  def __download_link(self, url, status=0):

    if (self.__now_count>8):
      print ('ddddddddddddddddddddddddddddddddddddddddd')
      raise NNN
    
    kod_log = self.__subd.add_log(self.__class__.__name__,'__download_link',1,f'--- начало, url={url}, status={status}')
    if kod_log !=0:
      print('ошибка при записи в лог из краулера.__download_link lvl=1, ошибка-',kod_log)
      return 20000 #работать не можем дальше, останавливаемся, нужно разбираться    

    
    
    
    if self.__now_recurs >= self.__setup_lim_recurs:
      print (f'    {datetime.now()}| cls_kr.DOWNLOAD_LINK(idO={self.__krregno}): достигли лимит по кол-ву рекурсий выходим.')
      return 'хватит по уровню рекурсии'
    isklu4enie = None #моя личная классификация исключений. один код может включать несколько реальных кодов ошибок. так что это группировочные некие коды
    errtxt = '' #для записи в БД текст ошибки. он будет появляться только если сработал эксепшн.

    def get_redirect_url(rr): #это для возвращения урла, на который идет редирект
      if rr.status_code in [301, 302, 303, 307, 308]:
        return rr.headers.get('location') #регистронезависимое извлечение заголовка
      return ''
    
    try:
      self.__now_count += 1 #попытка увеличивается независимо от того, будет ли гет успешным. может ссылка живая, но заблокирована по законодательству. получим таймаут или еще чего, но попытка увеличивается
      response = requests.get(url,timeout=(3, 10), allow_redirects=False)  # 3 сек. на подключение, 10 сек. на чтение, запрет редиректов, иначе не разобраться
      isklu4enie = 0   #если эксепшенов не было, то поставим в ноль. будет означать, что какойто код у нас имеется
      response.raise_for_status() # Метод raise_for_status() выбросит исключение HTTPError, если статус ответа указывает на ошибку (4xx или 5xx):
      kod_otveta=response.status_code
      kod_log = self.__subd.add_log(self.__class__.__name__,'__download_link',10,f' get(url) выполнен для: {url}. код ответа={kod_otveta}, всего посетили страниц: {self.__now_count}')
      if kod_log !=0:
        print('ошибка при записи в лог из краулера.__download_link lvl=10, ошибка-',kod_log)
        return 20000 #работать не можем дальше, останавливаемся, нужно разбираться
      
      kod_insert = self.__subd.ins_url(url, get_redirect_url(response), isklu4enie, kod_otveta, status, self.__setup_lim_recurs,self.__setup_lim_href,self.__setup_lim_size, len(response.content),errtxt,self.__now_recurs) #запись в бд
      if kod_insert == errorcode.ER_DUP_ENTRY:
        self.__urldubley += 1
        kod_log = self.__subd.add_log(self.__class__.__name__,'__download_link',20,f'обнаружена попытка задублировать строку. похоже, что ее мы уже обрабатывали раньше. выходим.')
        if kod_log !=0:
          print('ошибка при записи в лог из краулера.__download_link lvl=20, ошибка-',kod_log)
          return 20000 #работать не можем дальше, останавливаемся, нужно разбираться
        return 'дубль'
      elif self.__now_count >= self.__max_count:
        kod_log = self.__subd.add_log(self.__class__.__name__,'__download_link',30,f'Достигли предел настроек по общему количестсву: max={self.__max_count}, текущий счетчик: {self.__now_count}')
        if kod_log !=0:
          print('ошибка при записи в лог из Кrauler.__download_link lvl=30 ошибка-',kod_log)
          return 20000 #работать не можем дальше, останавливаемся, нужно разбираться
        return 'достигнут общий лимит'
      if kod_otveta == 200:
        kod_log = self.__subd.add_log(self.__class__.__name__,'__download_link',40,f'обрабатываю ответ 200, буду посылать в парсер')
        if kod_log !=0:
          print('ошибка при записи в лог из Кrauler.__download_link lvl=40 ошибка-',kod_log)
          return 20000 #работать не можем дальше, останавливаемся, нужно разбираться
        soup = BeautifulSoup(response.text, 'html.parser')
        self.__parse_links(soup, url)
      else:
        kod_log = self.__subd.add_log(self.__class__.__name__,'__download_link',50,f'Парсить не будем, т.к. не получил код 200: {url} (Status code: {kod_otveta})')
        if kod_log !=0:
          print('ошибка при записи в лог из Кrauler.__download_link lvl=50 ошибка-',kod_log)
          return 20000 #работать не можем дальше, останавливаемся, нужно разбираться
        
    except requests.exceptions.ConnectTimeout as e:
      isklu4enie = 1
      errtxt = e
    except requests.exceptions.ReadTimeout as e:
      isklu4enie = 2
      errtxt = e
    except requests.exceptions.Timeout as e:
      isklu4enie = 3
      errtxt = e
    except requests.exceptions.ConnectionError as e:
      isklu4enie = 1000
      errtxt = e
    except HTTPError as e:
      isklu4enie = 404
      errtxt = e
    except requests.RequestException as e: #если скачать страничку не получилось. ошибка вобще непонятная при получении request
      isklu4enie = 999
      errtxt = e
      print(f"  {datetime.now()}| cls_kr.DOWNLOAD_LINK(idO={self.__krregno}): ошибка 999 при попытке сделать GET непонятная: {e}")
    except Exception as e: #если ошибка вобще-вобще непонятная
      isklu4enie = 9999
      errtxt = e
      print(f"  {datetime.now()}| cls_kr.DOWNLOAD_LINK(idO={self.__krregno}): ошибка 9999 вобще-вобще непонятная: {e} nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn")
    
    if isklu4enie == None: # вобще такого никогда не должно быть
      print(f'  {datetime.now()}| cls_kr.DOWNLOAD_LINK(idO={self.__krregno}): вобще хрень какая-то, не должно такого быть. на выходе.')
      return 'error_HZ'
    elif isklu4enie == 0:
      return 'ok'  # если ноль, то значит мы сделали гет и он прошел норм и мы уже адрес записали в бд и сходили по рекурсии парсить
    else: #здесь формируем инсерт в том случае, если было какое-то исключение.
      kod_insert = self.__subd.ins_url(url, '', isklu4enie, 'NULL', status, self.__setup_lim_recurs,self.__setup_lim_href,self.__setup_lim_size, 0, errtxt,self.__now_recurs) #запись в бд
    #except Exception: #если скачать страничку не получилось вобще непонятно почему
      #print(f"  cls_kr.DOWNLOAD_LINK(idO={self.__krregno}): ошибка вобще ХЗ --{Exception.error}")
      #print(f"    cls_kr.DOWNLOAD_LINK(idO={self.__krregno}): переданный урл:{url}")
        # Delay to avoid overwhelming the server
    time.sleep(1)

  def __parse_links(self, soup, base_url):
    print(f'{datetime.now()}| cls_kr.PARSE_LINKS(idO={self.__krregno}): зашел')
    self.__now_recurs += 1
    for link in soup.find_all('a', href=True):
      absolute_url = urljoin(base_url, link['href']) #здесь могут быть еще параметры всякие в урле, нужно их выкусить
      p_url = urlsplit(absolute_url)
      absolute_url = urlunsplit((p_url.scheme, p_url.netloc, p_url.path, '', '')) #вот тут все выкусили из урла, все параметры и их значения. может и потом нужно их обрабатывать будет
      #if absolute_url not in self.__visited:
        #self.__download_link(absolute_url) # уходим в рекурсию
      #print('абсолютный urllllllll:',absolute_url,', base_url=',base_url, '. сам href=', link['href'])
      print (f"  {datetime.now()}| cls_kr.PARSE_LINKS(idO={self.__krregno})| base_url={base_url}: нашел такой:{absolute_url}, link={link['href']}")
      kod_rec = self.__download_link(absolute_url) # уходим в рекурсию
    print (f"  {datetime.now()}| cls_kr.PARSE_LINKS(idO={self.__krregno})| base_url={base_url}: вышел из рекурсии:{absolute_url} с кодом выхода {kod_rec}, сейчас буду брать след. адрес")
    self.__now_recurs -= 1


In [ ]:
resp = requests.get('https://biblio1.ru/test_diser1.html')

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
resp.raise_for_status()

In [ ]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
ll=[]

In [ ]:
for link in soup.find_all('a', href=True):
    print ('сам линк=',link)
    print('href=',link['href'])
    ll.append(link['href'])
    print('===')

print('-------------------')
ll